In [12]:
!python -m spacy download ca_core_news_trf

     ---------------------------------------- 0.0/457.1 MB ? eta -:--:--
     -------------------------------------- 0.0/457.1 MB 320.0 kB/s eta 0:23:49
     -------------------------------------- 0.1/457.1 MB 777.7 kB/s eta 0:09:48
     ---------------------------------------- 0.6/457.1 MB 5.0 MB/s eta 0:01:32
     ---------------------------------------- 1.4/457.1 MB 8.1 MB/s eta 0:00:57
     --------------------------------------- 2.4/457.1 MB 10.7 MB/s eta 0:00:43
     --------------------------------------- 3.5/457.1 MB 12.9 MB/s eta 0:00:36
     --------------------------------------- 4.8/457.1 MB 15.4 MB/s eta 0:00:30
      -------------------------------------- 6.4/457.1 MB 17.8 MB/s eta 0:00:26
      -------------------------------------- 8.4/457.1 MB 20.6 MB/s eta 0:00:22
      ------------------------------------- 10.6/457.1 MB 29.7 MB/s eta 0:00:16
     - ------------------------------------ 13.6/457.1 MB 43.7 MB/s eta 0:00:11
     - ------------------------------------ 15.

In [1]:
from typing import Tuple, List, Optional
import spacy
import numpy as np
from model_bàsic import build_and_compile_model_better

In [14]:
# Cargar el modelo de spaCy con integración de transformers para catalán
roberta_model = spacy.load("ca_core_news_trf")

def get_roberta_embedding(text):
    return np.mean(roberta_model(text)._.trf_data.last_hidden_layer_state.data[:-1], axis=0)

In [15]:
def map_pairs(
    sentence_pairs: List[Tuple[str, str, float]],
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        vector1 = get_roberta_embedding(sentence_1)
        vector2 = get_roberta_embedding(sentence_2)
        # Añadir a la lista
        
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [16]:
from importació_data import read_all_ts_data, reformat_data

# Get data
train, val, test = read_all_ts_data()

#Transform data
train_pairs, val_pairs, test_pairs = reformat_data(train, val, test)

In [17]:
mapped_train = map_pairs(train_pairs)
mapped_val = map_pairs(val_pairs)
mapped_test = map_pairs(test_pairs)

In [18]:
# Prepare the data
from importació_data import pair_list_to_x_y

X_train, y_train = pair_list_to_x_y(mapped_train)
X_val, y_val = pair_list_to_x_y(mapped_val)
X_test, y_test = pair_list_to_x_y(mapped_test)

In [ ]:
model = build_and_compile_model_better(768)

model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 768)       │    590,592 │ input_layer_6[0]… │
│                     │                   │            │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_6 (Lambda)   │ (None, 768)       │          0 │ dense_9[0][0],    │
│                     │                   │            │ dense_9[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 768)       │          0 │ lambda_6[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 128)       │     98,432 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 128)       │          0 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 16)        │      2,064 │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 16)        │          0 │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 1)         │         17 │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_7 (Lambda)   │ (None, 1)         │          0 │ dense_12[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 691,105 (2.64 MB)

 Trainable params: 691,105 (2.64 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(64)

In [ ]:
train_dataset

<_BatchDataset element_spec=((TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), TensorSpec(shape=(None, 768), dtype=tf.float32, name=None)), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>

In [ ]:
model.fit(train_dataset, epochs=100, validation_data=val_dataset)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.7260 - val_loss: 0.7254
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7177 - val_loss: 0.7222
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7147 - val_loss: 0.7194
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7070 - val_loss: 0.7190
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6909 - val_loss: 0.7211
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6802 - val_loss: 0.7243
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6650 - val_loss: 0.7286
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6546 - val_loss: 0.7336
Epoch 9/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6518 - val_loss: 0.7378
Epoch 10/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6308 - val_loss: 0.7399
Epoch 11/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6252 - val_loss: 0.7531
Epoch 12/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0

In [ ]:
from scipy.stats import pearsonr

x_test, y_test = pair_list_to_x_y(mapped_test)
def compute_pearson(x_, y_, model):
    # Get predictions for the model
    y_pred = model.predict(x_)
    # Compute pearson correlation
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation

In [ ]:
print(f"Correlación de Pearson (train): {compute_pearson(X_train, y_train, model)}")
print(f"Correlación de Pearson (validation): {compute_pearson(X_val, y_val, model)}")
print(f"Correlación de Pearson (test): {compute_pearson(X_test, y_test, model)}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Correlación de Pearson (train): 0.7361164836771393
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson (validation): 0.15813986630961854
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson (test): 0.23395487116758615
